In [1]:
!pip install --ignore-installed -q pyspark==3.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 22.4 MB/s eta 0:00:00


In [2]:
import os
import sys
import requests
from operator import add

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)



/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:

# requesting Spark context
#sc = SparkContext(appName="HW1_Task1")


In [3]:
#Exception Handling and removing wrong datalines
def isfloat(value):
    try:
        float(value)
        return True

    except:
         return False

In [4]:
#Function - Cleaning
#For example, remove lines if they don’t have 16 values and
# checking if the trip distance and fare amount is a float number
# checking if the trip duration is more than a minute, trip distance is more than 0.1 miles,
# fare amount and total amount are more than 0 dollars
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[4])> 60 and float(p[5])>0 and float(p[11])> 0 and float(p[16])> 0):
                return p


In [5]:
# Upload the data
# At this point you have to upload your data to google colab
!wget -O /content/taxi-data-sorted-verysmall.csv https://storage.googleapis.com/met-cs-777-data/taxi-data-sorted-verysmall.csv

--2023-09-18 19:56:07--  https://storage.googleapis.com/met-cs-777-data/taxi-data-sorted-verysmall.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.6.207, 172.217.212.207, 74.125.124.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.6.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1921037 (1.8M) [text/csv]
Saving to: ‘/content/taxi-data-sorted-verysmall.csv’

/content/taxi-data- 100%[===================>]   1.83M  --.-KB/s    in 0.02s   

2023-09-18 19:56:08 (84.2 MB/s) - ‘/content/taxi-data-sorted-verysmall.csv’ saved [1921037/1921037]



In [6]:
pwd

'/content'

# New Section

In [7]:
ls -la

total 1896
drwxr-xr-x 1 root root    4096 Sep 18 19:56 ./
drwxr-xr-x 1 root root    4096 Sep 18 19:54 ../
drwxr-xr-x 4 root root    4096 Sep 15 13:21 .config/
drwxr-xr-x 1 root root    4096 Sep 15 13:22 sample_data/
-rw-r--r-- 1 root root 1921037 May 13 15:47 taxi-data-sorted-verysmall.csv


In [8]:
# Set your file path here
path="file:///content/"

# You have 1 files

testFile= path + "taxi-data-sorted-verysmall.csv"


testDataFrame = sqlContext.read.format('csv').options(header='false', inferSchema='true',  sep =",").load(testFile)

testDataFrame.show()

+--------------------+--------------------+-------------------+-------------------+---+----+----------+---------+----------+---------+----+----+----+----+----+----+----+
|                 _c0|                 _c1|                _c2|                _c3|_c4| _c5|       _c6|      _c7|       _c8|      _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|
+--------------------+--------------------+-------------------+-------------------+---+----+----------+---------+----------+---------+----+----+----+----+----+----+----+
|07290D3599E7A0D62...|E7750A37CAB07D0DF...|2013-01-01 00:00:00|2013-01-01 00:02:00|120|0.44|-73.956528|40.716976| -73.96244|40.715008| CSH| 3.5| 0.5| 0.5| 0.0| 0.0| 4.5|
|22D70BF00EEB0ADC8...|3FF2709163DE7036F...|2013-01-01 00:02:00|2013-01-01 00:02:00|  0| 0.0|       0.0|      0.0|       0.0|      0.0| CSH|27.0| 0.0| 0.5| 0.0| 0.0|27.5|
|0EC22AAF491A8BD91...|778C92B26AE78A9EB...|2013-01-01 00:01:00|2013-01-01 00:03:00|120|0.71|-73.973145|40.752827|-73.965897|40.760445| CSH| 4.0| 0.5| 

In [9]:
testRDD = testDataFrame.rdd.map(tuple)
testRDD.take(1)

[('07290D3599E7A0D62097A346EFCC1FB5',
  'E7750A37CAB07D0DFF0AF7E3573AC141',
  '2013-01-01 00:00:00',
  '2013-01-01 00:02:00',
  120,
  0.44,
  -73.956528,
  40.716976,
  -73.96244,
  40.715008,
  'CSH',
  3.5,
  0.5,
  0.5,
  0.0,
  0.0,
  4.5)]

In [10]:

# calling isfloat and correctRows functions to cleaning up data
taxilinesCorrected = testRDD.filter(correctRows)
taxilinesCorrected.take(1)

[('07290D3599E7A0D62097A346EFCC1FB5',
  'E7750A37CAB07D0DFF0AF7E3573AC141',
  '2013-01-01 00:00:00',
  '2013-01-01 00:02:00',
  120,
  0.44,
  -73.956528,
  40.716976,
  -73.96244,
  40.715008,
  'CSH',
  3.5,
  0.5,
  0.5,
  0.0,
  0.0,
  4.5)]

In [11]:
x = taxilinesCorrected.map(lambda x: (x[0], x[1]))
count = x.distinct().countByKey()
top10_1 = sorted(count.items(), key=lambda x: -x[1])[:10]
results_1 = sc.parallelize(top10_1)
results_1.collect()

[('F899990AB4A6582D6A5F21265725216E', 2),
 ('582CAF46446FA03320E5178E7BEC1E86', 2),
 ('DE85355130669A29081912E9AA6CCD5D', 2),
 ('07290D3599E7A0D62097A346EFCC1FB5', 1),
 ('0EC22AAF491A8BD91F279350C2B010FD', 1),
 ('1390FB380189DF6BBFDA4DC847CAD14F', 1),
 ('3B4129883A1D05BE89F2C929DE136281', 1),
 ('468244D1361B8A3EB8D206CC394BC9E9', 1),
 ('5F78CC6D4ECD0541B765FECE17075B6F', 1),
 ('6BA29E9A69B10F218C1509BEDD7410C2', 1)]

In [39]:
# create calculated field for earned money per minute and a count field
def calculated(rdd):
    new_field =  rdd[16]/(rdd[4]/60)
    return rdd + (new_field,) + (1,)
newrdd = taxilinesCorrected.map(calculated)
sum = newrdd.map(lambda x: (x[1], (x[17], x[18]))).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
average = sum.map(lambda x: (x[0], x[1][0] / x[1][1]))
top10_2 = average.top(10, key=lambda x: x[1])
results_2 = sc.parallelize(top10_2)
results_2.collect()

[('371910291B554B9821CA346094832F60', 7.025),
 ('911299BE5F3FF1DFD6E3F4BFEACB6E26', 5.571428571428571),
 ('3C80E82705CE0E0A15D12ACD455C0B58', 4.535714285714286),
 ('8C25B80C13FE9A642D82EE6E5B157C39', 4.5),
 ('B72DEE45763A098CE0434638D9DAD7B0', 4.428571428571429),
 ('74D1BD03A75AF44B8293DB50F6943570', 4.355263157894737),
 ('90253C37D48D6BDCA8E0833804902306', 4.0675675675675675),
 ('08DB3F9FCF01530D6F7E70EB88C3AE5B', 3.8333333333333335),
 ('8C72DB8F9EB81A2D4F923C909D93FFDB', 3.75),
 ('904DB583CC701A7BE6F483CDB2D470D5', 3.74375)]